# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [137]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sqlalchemy import create_engine
import joblib

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import time

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

%matplotlib inline

[nltk_data] Downloading package punkt to /Users/boubou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/boubou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/boubou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [141]:
# load data from database
engine = create_engine('sqlite:///message_features.db')
df = pd.read_sql_table('messages', engine)  
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
X = df["message"]
Y = df.drop(labels=["id","message", "original", "genre"], axis=1)

In [74]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [75]:
print(Y.shape)
Y.head()

(26216, 36)


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
Y.columns.values.tolist()

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [76]:
# Check column value: its seems "related" and "child_alone" are not fully binary column
Y.describe()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [77]:
for i,v in enumerate(Y):
    num = Y[v].unique()
    if len(num)!=2:
        print(v)
        print(num)

related
[1 0 2]
child_alone
[0]


In [79]:
# Drop 
Y = Y.drop(labels=["child_alone"], axis=1)
Y["related"] = Y["related"].apply(lambda x: 1 if x > 0 else 0)

In [80]:
#Check
for i,v in enumerate(Y):
    num = Y[v].unique()
    if len(num)!=2:
        print(v)
        print(num)

In [51]:
#Percentage of each classes
Y.sum(axis=0)/Y.shape[0]

related                   0.773650
request                   0.170659
offer                     0.004501
aid_related               0.414251
medical_help              0.079493
medical_products          0.050084
search_and_rescue         0.027617
security                  0.017966
military                  0.032804
child_alone               0.000000
water                     0.063778
food                      0.111497
shelter                   0.088267
clothing                  0.015449
money                     0.023039
missing_people            0.011367
refugees                  0.033377
death                     0.045545
other_aid                 0.131446
infrastructure_related    0.065037
transport                 0.045812
buildings                 0.050847
electricity               0.020293
tools                     0.006065
hospitals                 0.010795
shops                     0.004577
aid_centers               0.011787
other_infrastructure      0.043904
weather_related     

### 2. Write a tokenization function to process your text data

In [81]:
def tokenize(text):
    
    # Normalise by setting to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Create tokens 
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    # Lemmatise words
    clean_tokens = [WordNetLemmatizer().lemmatize(w) for w in tokens]

    return clean_tokens

In [82]:
tokenize(X.iloc[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [83]:
basic_model = LogisticRegression(random_state=42)

basic_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(basic_model, n_jobs=-1))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [84]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

In [85]:
Y_train.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
16528,1,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
19112,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18834,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9228,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
14701,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
Y_train.describe()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,...,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.000000,19662.00000,19662.000000
mean,0.765385,0.169820,0.004832,0.412878,0.079341,0.050147,0.027159,0.018411,0.033618,0.063473,...,0.011799,0.042315,0.276218,0.081274,0.090327,0.011037,0.094751,0.020700,0.05147,0.192452
std,0.423769,0.375484,0.069344,0.492364,0.270277,0.218255,0.162551,0.134436,0.180249,0.243817,...,0.107985,0.201312,0.447137,0.273262,0.286656,0.104476,0.292879,0.142381,0.22096,0.394236
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [64]:
#Y_train.info()

In [87]:
basic_start_time = time.time()
basic_pipeline.fit(X_train,Y_train)
basic_end_time = time.time() - basic_start_time
print("the training lasts for {} minutes".format(basic_end_time/60))

the training lasts for 1.799617063999176 minutes


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [88]:
Y_pred = basic_pipeline.predict(X_test)

In [110]:
Y_pred.shape
#Y_pred[:,1].tolist()

(6554, 35)

In [96]:
output_classes = Y.columns.values.tolist()
output_classes[0:5]

['related', 'request', 'offer', 'aid_related', 'medical_help']

In [102]:
#Y_test['related'].values.tolist()

In [113]:
for class_idx,class_name in enumerate(Y):
    print("Column {}: {}".format(class_idx, class_name))
    
    y_true = Y_test[class_name].values.tolist()
    y_pred = Y_pred[:,class_idx].tolist()
    target_names = ['is_{}'.format(class_name), 'is_not_{}'.format(class_name)]
    print(classification_report(y_true, y_pred, target_names=target_names))

Column 0: related
                precision    recall  f1-score   support

    is_related       0.73      0.40      0.52      1509
is_not_related       0.84      0.96      0.90      5045

     micro avg       0.83      0.83      0.83      6554
     macro avg       0.79      0.68      0.71      6554
  weighted avg       0.82      0.83      0.81      6554

Column 1: request
                precision    recall  f1-score   support

    is_request       0.91      0.98      0.94      5419
is_not_request       0.83      0.52      0.64      1135

     micro avg       0.90      0.90      0.90      6554
     macro avg       0.87      0.75      0.79      6554
  weighted avg       0.89      0.90      0.89      6554

Column 2: offer
              precision    recall  f1-score   support

    is_offer       1.00      1.00      1.00      6531
is_not_offer       0.00      0.00      0.00        23

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      655

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

    is_death       0.96      1.00      0.98      6252
is_not_death       0.90      0.24      0.38       302

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.93      0.62      0.68      6554
weighted avg       0.96      0.96      0.95      6554

Column 17: other_aid
                  precision    recall  f1-score   support

    is_other_aid       0.88      0.99      0.93      5687
is_not_other_aid       0.60      0.12      0.19       867

       micro avg       0.87      0.87      0.87      6554
       macro avg       0.74      0.55      0.56      6554
    weighted avg       0.84      0.87      0.83      6554

Column 18: infrastructure_related
                               precision    recall  f1-score   support

    is_infrastructure_related       0.93      1.00      0.96      6097
is_not_infrastructure_related       0.72      0.04      0.07       457

                    micro avg       0.93      0.93  

### 6. Improve your model
Use grid search to find better parameters. 

In [116]:
basic_pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a209819d8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
             intercept_scaling=1, max_iter=100, multi_class='warn',
             n_jobs=None, penalty='l2', random_state=42, solver='warn',
             tol=0.0001, verbose=0, warm_start=False),
              n_jobs=-1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<c

In [119]:
parameters = {'clf__estimator__C': np.linspace(1,10,3),
             'clf__estimator__penalty': ['l1', 'l2']}

gridsearch_pipeline = GridSearchCV(basic_pipeline, param_grid=parameters, scoring='precision_samples', cv = 3)

gridsearch_start_time = time.time()
gridsearch_pipeline.fit(X_train,Y_train)
gridsearch_end_time = time.time() - gridsearch_start_time
print("the training lasts for {} minutes".format(gridsearch_end_time/60))

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
Y_pred_GS = gridsearch_pipeline.fit(X_test)

In [ ]:
for class_idx,class_name in enumerate(Y):
    print("Column {}: {}".format(class_idx, class_name))
    
    y_true = Y_test[class_name].values.tolist()
    y_pred = Y_pred_GS[:,class_idx].tolist()
    target_names = ['is_{}'.format(class_name), 'is_not_{}'.format(class_name)]
    print(classification_report(y_true, y_pred, target_names=target_names))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [121]:
forest = RandomForestClassifier(random_state=42)

forest_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(forest, n_jobs=-1))
    ])

In [129]:
#forest_pipeline.get_params()

In [130]:
improve_start_time = time.time()
forest_pipeline.fit(X_train,Y_train)
improve_end_time = time.time() - improve_start_time
print("the training lasts for {} minutes".format(improve_end_time/60))

the training lasts for 2.314076781272888 minutes


In [131]:
Y_pred_RF = forest_pipeline.predict(X_test)

In [132]:
for class_idx,class_name in enumerate(Y):
    print("Column {}: {}".format(class_idx, class_name))
    
    y_true = Y_test[class_name].values.tolist()
    y_pred = Y_pred_RF[:,class_idx].tolist()
    target_names = ['is_{}'.format(class_name), 'is_not_{}'.format(class_name)]
    print(classification_report(y_true, y_pred, target_names=target_names))

Column 0: related
                precision    recall  f1-score   support

    is_related       0.61      0.45      0.52      1509
is_not_related       0.85      0.92      0.88      5045

     micro avg       0.81      0.81      0.81      6554
     macro avg       0.73      0.68      0.70      6554
  weighted avg       0.79      0.81      0.80      6554

Column 1: request
                precision    recall  f1-score   support

    is_request       0.89      0.98      0.93      5419
is_not_request       0.81      0.43      0.56      1135

     micro avg       0.88      0.88      0.88      6554
     macro avg       0.85      0.71      0.75      6554
  weighted avg       0.88      0.88      0.87      6554

Column 2: offer
              precision    recall  f1-score   support

    is_offer       1.00      1.00      1.00      6531
is_not_offer       0.00      0.00      0.00        23

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      655

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                  precision    recall  f1-score   support

    is_transport       0.96      1.00      0.98      6241
is_not_transport       0.72      0.12      0.20       313

       micro avg       0.96      0.96      0.96      6554
       macro avg       0.84      0.56      0.59      6554
    weighted avg       0.95      0.96      0.94      6554

Column 20: buildings
                  precision    recall  f1-score   support

    is_buildings       0.96      1.00      0.98      6232
is_not_buildings       0.66      0.12      0.21       322

       micro avg       0.95      0.95      0.95      6554
       macro avg       0.81      0.56      0.59      6554
    weighted avg       0.94      0.95      0.94      6554

Column 21: electricity
                    precision    recall  f1-score   support

    is_electricity       0.98      1.00      0.99      6416
is_not_electricity       1.00      0.01      0.03       138

         micro avg       0.98      0.98      0.98      6554
         mac

### 9. Export your model as a pickle file

In [138]:
pipeline_to_export = basic_pipeline
joblib.dump(pipeline_to_export, "model.pkl")

['model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [144]:
model = joblib.load("model.pkl")

In [146]:
model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a209819d8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
             intercept_scaling=1, max_iter=100, multi_class='warn',
             n_jobs=None, penalty='l2', random_state=42, solver='warn',
             tol=0.0001, verbose=0, warm_start=False),
              n_jobs=-1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<c